<a href="https://colab.research.google.com/github/Richard3012/Event_Management_Mad/blob/master/ML_StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit keras pandas numpy matplotlib

In [ ]:
!pip install streamlit pyngrok

In [ ]:
!ngrok authtoken 2tXALRPVNUueZKSdKbG5i6s4uOU_3VtucNeHfMYN5qsyb53ez

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# from pyngrok import ngrok
# ngrok.set_auth_token("2tXALRPVNUueZKSdKbG5i6s4uOU_3VtucNeHfMYN5qsyb53ez")
# public_url = ngrok.connect(8501)
# print("Access the Streamlit app here:", public_url)

In [ ]:
from pyngrok import ngrok
import os

# 🔹 Kill any previous Streamlit processes (Linux/macOS)
os.system("pkill -f streamlit")

# 🔹 If on Windows, kill Streamlit manually (uncomment if needed)
# os.system("taskkill /IM streamlit.exe /F")

# 🔹 Start Streamlit
os.system("streamlit run /content/app.py --server.port 8501 --server.headless true &")

# 🔹 Authenticate ngrok (Replace with your actual token)
NGROK_AUTH_TOKEN = "2tXALRPVNUueZKSdKbG5i6s4uOU_3VtucNeHfMYN5qsyb53ez"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 🔹 Create public URL
public_url = ngrok.connect(8501)
print("Access the Streamlit app here:", public_url)


Access the Streamlit app here: NgrokTunnel: "https://85eb-35-225-255-0.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import os
print(os.path.getsize("/content/Stock_Predictions_Model.keras"))


419706


In [ ]:
from keras.models import load_model

model = load_model("/content/Stock_Predictions_Model.keras")



In [ ]:
!pip install alpha_vantage

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from sklearn.preprocessing import MinMaxScaler
import numpy as np

API_KEY = "CX9KDAMOZYZ1HQ99"  # Replace with your actual API key

def get_stock_data(stock):
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='full')
    print(data.head())  # Check if data is retrieved correctly
    return data

# Fetch stock data before using it
data = get_stock_data("GOOG")  # Replace "GOOG" with any stock symbol

# Check if data is retrieved properly
if data is None or data.empty:
    raise ValueError("⚠️ Warning: No data retrieved! Check API response or API Key.")

# Select 'close' prices and reverse order (oldest first)
data = data[['4. close']].iloc[::-1]

# Scale data
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data)

# Create training sequences
x_train, y_train = [], []
for i in range(100, len(data_scaled)):
    x_train.append(data_scaled[i-100:i])  # Use last 100 days as input
    y_train.append(data_scaled[i, 0])  # Predict next day's price

x_train, y_train = np.array(x_train), np.array(y_train)

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")


            1. open  2. high  3. low  4. close   5. volume
date                                                      
2025-02-24  183.800   185.09  180.88    181.19  18734014.0
2025-02-21  187.290   187.47  181.13    181.58  19520782.0
2025-02-20  186.500   187.12  184.60    186.64  12063807.0
2025-02-19  186.185   187.36  185.50    187.13  13120465.0
2025-02-18  187.440   187.78  183.58    185.80  19796028.0
x_train shape: (2645, 100, 1), y_train shape: (2645,)


In [ ]:
def get_stock_data(stock):
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='full')
    print(data.head())
    return data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Select 'close' prices and reverse order (oldest first)
data = data[['4. close']].iloc[::-1]

# Scale data
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data)

# Create training sequences
x_train, y_train = [], []
for i in range(100, len(data_scaled)):
    x_train.append(data_scaled[i-100:i])  # Use last 100 days as input
    y_train.append(data_scaled[i, 0])  # Predict next day's price

x_train, y_train = np.array(x_train), np.array(y_train)


In [ ]:
# Train the Model
model.fit(x_train, y_train, epochs=20, batch_size=32)

# Save the Model
model.save("Stock_Predictions_Model.keras")


Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - loss: 0.0269
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - loss: 0.0031
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - loss: 0.0027
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - loss: 0.0028
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - loss: 0.0019
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - loss: 0.0014
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - loss: 0.0015
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - loss: 0.0017
Epoch 9/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - loss: 0.0018
Epoch 10/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - loss: 9.5187e-04
Epoch 11/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - loss: 0.0011
Epoch 12/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - loss: 0.0010
Epoch 13/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - loss: 0.0016
Epoch 14/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - loss: 0.0013
Epoch 15/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - los

In [ ]:
# Import necessary modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Define Model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(100, 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=1)  # Output layer
])

# Compile Model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display Model Summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100, 50)             │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 50)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(data.columns)


Index(['4. close'], dtype='object')


In [ ]:
for col in data.keys():
    print(col)


4. close
